In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
import umap.umap_ as umap
import os

In [20]:
read_dir = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/morphseq_analyses/lmx1b"
figure_dir = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/slides/20240207/"
master_df_ref = pd.read_csv(os.path.join(read_dir, "morphseq_df.csv"), index_col=0)
hooke_latent_df = pd.read_csv(os.path.join(read_dir, "hooke_latent_df_clean.csv"))
morph_df = pd.read_csv(os.path.join(read_dir, "morph_latent_df_clean.csv"))

#### Can hooke latent space predict distance morphological distance from WT reference? 

In [6]:
from sklearn.neural_network import MLPRegressor

n_train = 100
np.random.seed(314)
option_vec = master_df_ref.index
train_indices = np.random.choice(option_vec, n_train, replace=False)
test_indices = np.asarray([i for i in option_vec if i not in train_indices])

# get test and train datasets
Y_train = master_df_ref.loc[train_indices, "wik_dist_morph"]
nan_train_indices = np.isnan(Y_train)

Y_train = Y_train[~nan_train_indices]
Y_test = master_df_ref.loc[test_indices, "wik_dist_morph"]
nan_test_indices = np.isnan(Y_test)
Y_test = Y_test[~nan_test_indices]

X_train = master_df_ref.loc[train_indices, ["UMAP_hooke_00_3", "UMAP_hooke_01_3", "UMAP_hooke_02_3"]]
X_train = X_train.loc[~nan_train_indices, :]

X_test = master_df_ref.loc[test_indices, ["UMAP_hooke_00_3", "UMAP_hooke_01_3", "UMAP_hooke_02_3"]]
X_test = X_test.loc[~nan_test_indices, :]

In [ ]:
# fit model
regr_dist = MLPRegressor(random_state=1, max_iter=5000, hidden_layer_sizes=(5000,))
regr_dist.fit(X_train, Y_train)

# get model scores
reg_score_train = regr_dist.score(X_train, Y_train)
reg_score_test = regr_dist.score(X_test, Y_test)

print(reg_score_train)
print(reg_score_test)

In [9]:
Y_test_pd = regr_dist.predict(X_test)

res_df_test = pd.DataFrame(Y_test_pd[:, np.newaxis], columns=["morph_wik_dist_pd"])
res_df_test["morph_wik_dist_actual"] = Y_test.to_numpy()

pert = master_df_ref.loc[test_indices, ["master_perturbation"]].copy()
pert = pert.loc[~nan_test_indices, :].to_numpy()
                      
                      
res_df_test["master_perturbation"] = pert

res_df_test.head(6)

,morph_wik_dist_pd,morph_wik_dist_actual,master_perturbation
0,2.022837,2.209699,lmx1b
1,0.675039,0.873535,wik-inj-ctrl
2,2.067292,2.068623,lmx1b
3,0.667817,0.755843,wik-inj-ctrl
4,1.886633,2.038143,lmx1b
5,2.100224,1.918916,lmx1b


In [30]:
tbxta_bool = (res_df_test["master_perturbation"] == "tbxta").to_numpy()
lmx_bool = (res_df_test["master_perturbation"] == "lmx1b").to_numpy()
ctrl_bool = (res_df_test["master_perturbation"] == "wik-inj-ctrl").to_numpy()

all_bool = tbxta_bool | lmx_bool | ctrl_bool

fig = px.scatter(res_df_test.loc[all_bool], x="morph_wik_dist_pd", y="morph_wik_dist_actual", color="master_perturbation",
                 opacity =0.8,
                 labels={"morph_wik_dist_actual": "actual distance from WT reference",
                         "morph_wik_dist_pd": "predicted distance from WT reference",
                         "master_perturbation":"perturbation"})

fig.update_traces(marker=dict(size=10,
                              line=dict(width=1,
                                        color='DarkSlateGrey')))

ref_line = np.linspace(-1, 15)
fig.add_scatter(
        x=ref_line, 
        y=ref_line, 
        mode='lines', 
        line_color='gray', 
        showlegend=False,
        line=dict(width=1, dash='dot')
    )

                  
fig.show()
# save
fig.write_image(os.path.join(figure_dir, "lmx_distance_prediction_plot.png"), scale=4)

Switch up the plotting style

In [50]:
import plotly.graph_objects as go

label_dict = {"morph_wik_dist_actual": "actual distance from WT reference",
                         "morph_wik_dist_pd": "predicted distance from WT reference",
                         "master_perturbation":"perturbation"}
pastel_red = "rgb(251,180,174)"

color_dict = dict({"lmx1b" :"#EF553B" , "wik":"black", "wik-inj-ctrl":"#636EFA", "tbxta":"#00CC96"})
color_vec = np.asarray([color_dict[p] for p in res_df_test.loc[:, "master_perturbation"].tolist()])

# just the line
fig = go.Figure()

ref_line = np.linspace(-1, 15)
fig.add_scatter(
        x=ref_line, 
        y=ref_line, 
        mode='lines', 
        line_color='gray', 
        showlegend=False,
        line=dict(width=2, dash='dot')
    )


fig.update_layout(
    xaxis_title="predicted distance from WT reference",
    yaxis_title="actual distance from WT reference",
    legend_title="perturbation",
    font=dict(
        family="Arial",
        size=14,
        color="Black"
    )
)

fig.add_scatter(x=res_df_test.loc[tbxta_bool, "morph_wik_dist_pd"], 
                y=res_df_test.loc[tbxta_bool, "morph_wik_dist_actual"], 
                mode="markers",
                marker=dict(size=10, opacity=0, color=color_vec[tbxta_bool],
                                      ),
                name="wik-inj-ctrl"
               )

fig.write_image(os.path.join(figure_dir, "lmx_distance_prediction_plot00.png"), scale=4)

#######
# add tbxta and inj-ctrl

fig = go.Figure()

ref_line = np.linspace(-1, 15)
fig.add_scatter(
        x=ref_line, 
        y=ref_line, 
        mode='lines', 
        line_color='gray', 
        showlegend=False,
        line=dict(width=2, dash='dot')
    )


fig.update_layout(
    xaxis_title="predicted distance from WT reference",
    yaxis_title="actual distance from WT reference",
    legend_title="perturbation",
    font=dict(
        family="Arial",
        size=14,
        color="Black"
    )
)

fig.add_scatter(x=res_df_test.loc[tbxta_bool, "morph_wik_dist_pd"], 
                y=res_df_test.loc[tbxta_bool, "morph_wik_dist_actual"], 
                mode="markers",
                marker=dict(size=10, opacity=1, color=color_vec[tbxta_bool],
                                      line=dict(width=0.8,
                                        color='DarkSlateGrey')),
                name="tbxta",
                showlegend=True
               )

fig.add_scatter(x=res_df_test.loc[ctrl_bool, "morph_wik_dist_pd"], 
                y=res_df_test.loc[ctrl_bool, "morph_wik_dist_actual"], 
                mode="markers",
                marker=dict(size=10, opacity=1, color=color_vec[ctrl_bool],
                                      line=dict(width=0.8,
                                        color='DarkSlateGrey')),
                name="wik-inj-ctrl"
               )



fig.write_image(os.path.join(figure_dir, "lmx_distance_prediction_plot01.png"), scale=4)
     
fig.add_scatter(x=res_df_test.loc[lmx_bool, "morph_wik_dist_pd"], 
                y=res_df_test.loc[lmx_bool, "morph_wik_dist_actual"], 
                mode="markers",
                marker=dict(size=10.5, opacity=1, color=color_vec[lmx_bool],
                                      line=dict(width=1.3,
                                        color='DarkSlateGrey')),
                name="lmx1b"
               )


fig.write_image(os.path.join(figure_dir, "lmx_distance_prediction_plot02.png"), scale=4)
                            
fig.show()
# # # save
# # fig.write_image(os.path.join(figure_dir, "lmx_distance_prediction_plot.png"), scale=4)

### Latent space plots